In [ ]:
#imports

import numpy as np
import pandas as pd
from google.colab import drive
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.metrics import precision_score
import nltk
import random
nltk.download('stopwords')

from nltk.corpus import stopwords
import string
stopset = stopwords.words('english') + list(string.punctuation)
nltk.download('punkt')
import re

import io


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#!pip install transformers sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 6.8 MB/s 
     |████████████████████████████████| 85 kB 6.1 MB/s 
     |████████████████████████████████| 182 kB 59.6 MB/s 
     |████████████████████████████████| 7.6 MB 61.9 MB/s 
     |████████████████████████████████| 1.3 MB 74.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=9cddcb6ebc44ebd8fda96c42643efa10f51a8459b2f38a8316bd1e4946685e92
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


In [ ]:
#more imports

import torch
import transformers as ppb

In [ ]:
#mount drive + load data

drive.mount('/content/drive', force_remount=True)
nyt = pd.read_csv('/content/drive/MyDrive/NYT.csv')

Mounted at /content/drive


In [ ]:
#split into train/val/test sets

years = [int(i[-4:]) for i in nyt['date']]
nyt['year'] = years
nyt_train = nyt[nyt['year'] <= 2020]
nyt_val = nyt[nyt['year'] == 2021]
nyt_test = nyt[nyt['year'] == 2022]

In [ ]:
#train answers

val_index_update = []
train_answers = [k for j in [list(eval(i)) for i in nyt_train['answer']] for k in j]

for i in nyt_test.index:
  #print(i)
  temp_answer = list(eval(nyt_test['answer'][i]))
  if np.mean([bool(m in train_answers) for m in temp_answer]) > 0:
    val_index_update.append(i)
  else:
    continue


10024
10025
10026
10027
10028
10029
10030
10031
10032
10033
10034
10035
10036
10037
10038
10039
10040
10041
10042
10043
10044
10045
10046
10047
10048
10049
10050
10051
10052
10053
10054
10055
10056
10057
10058
10059
10060
10061
10062
10063
10064
10065
10066
10067
10068
10069
10070
10071
10072
10073
10074
10075
10076
10077
10078
10079
10080
10081
10082
10083
10084
10085
10086
10087
10088
10089
10090
10091
10092
10093
10094
10095
10096
10097
10098
10099
10100
10101
10102
10103
10104
10105
10106
10107
10108
10109
10110
10111
10112
10113
10114
10115
10116
10117
10118
10119
10120
10121
10122
10123
10124
10125
10126
10127
10128
10129
10130
10131
10132
10133
10134
10135
10136
10137
10138
10139
10140
10141
10142
10143
10144
10145
10146
10147
10148
10149
10150
10151
10152
10153
10154
10155
10156
10157
10158
10159
10160
10161
10162
10163
10164
10165
10166
10167
10168
10169
10170
10171
10172
10173
10174
10175
10176
10177
10178
10179
10180
10181
10182
10183
10184
10185
10186
10187
10188
10189
1019

In [ ]:
#validation answers, train clues, validation clues

val_answers = [k for j in [list(eval(i)) for i in nyt_val['answer']] for k in j]
train_clues = [k for j in [list(eval(i)) for i in nyt_train['clue']] for k in j]
val_clues = [k for j in [list(eval(i)) for i in nyt_val['clue']] for k in j]

In [ ]:
#make arrays 

train_answers = np.array(train_answers)
val_answers = np.array(val_answers)
train_clues = np.array(train_clues)
val_clues = np.array(val_clues)

In [ ]:
#load models + create final SentenceTransformer Model

from sentence_transformers import SentenceTransformer, models
from transformers import AutoConfig

word_model = SentenceTransformer('all-MiniLM-L6-v2')
pooling_model = models.Pooling(128)

config = AutoConfig.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
config.hidden_dropout_prob = 0.2
config.attention_probs_dropout_prob = 0.2
config.num_hidden_layers = 8

model = SentenceTransformer(modules=[word_model, pooling_model])

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

In [ ]:
#load clue/answer pairs as inputs for training

from sentence_transformers import InputExample

train_examples = [InputExample(texts=[train_clues[i], train_answers[i]]) for i in range(len(train_clues))]

In [ ]:
#load data in batches

from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=384)

In [ ]:
#configure loss for model

from sentence_transformers import losses

train_loss = losses.MultipleNegativesRankingLoss(model)

In [ ]:
#fit model using loaded data + los

from sentence_transformers import SentenceTransformer, models


torch.cuda.empty_cache()
#pool = model.start_multi_process_pool('cuda')
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=50, warmup_steps=100) 



Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2100 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2100 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2100 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2100 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/2100 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2100 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2100 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2100 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2100 [00:00<?, ?it/s]

In [ ]:
#Compute training embeddings
emb = []
for i in range(len(train_answers)):
  print(i)
  emb.append(model.encode([train_clues[i],train_answers[i]]))


Streaming output truncated to the last 5000 lines.
801218
801219
801220
801221
801222
801223
801224
801225
801226
801227
801228
801229
801230
801231
801232
801233
801234
801235
801236
801237
801238
801239
801240
801241
801242
801243
801244
801245
801246
801247
801248
801249
801250
801251
801252
801253
801254
801255
801256
801257
801258
801259
801260
801261
801262
801263
801264
801265
801266
801267
801268
801269
801270
801271
801272
801273
801274
801275
801276
801277
801278
801279
801280
801281
801282
801283
801284
801285
801286
801287
801288
801289
801290
801291
801292
801293
801294
801295
801296
801297
801298
801299
801300
801301
801302
801303
801304
801305
801306
801307
801308
801309
801310
801311
801312
801313
801314
801315
801316
801317
801318
801319
801320
801321
801322
801323
801324
801325
801326
801327
801328
801329
801330
801331
801332
801333
801334
801335
801336
801337
801338
801339
801340
801341
801342
801343
801344
801345
801346
801347
801348
801349
801350
801351
801352
8013

In [ ]:
#save clue/answer embeddings to array

clue_embed = np.array([emb[j][0] for j in range(len(emb))])
answer_embed = np.array([emb[j][1] for j in range(len(emb))])

In [ ]:
#export csv to drive

pd.DataFrame(clue_embed).to_csv('/content/drive/MyDrive/clueembed2.csv', index=False)
pd.DataFrame(answer_embed).to_csv('/content/drive/MyDrive/answerembed2.csv', index=False)

In [ ]:
#export data splits to drive

pd.DataFrame(nyt_train).to_csv('/content/drive/MyDrive/nyttrain2.csv', index=False)
pd.DataFrame(nyt_val).to_csv('/content/drive/MyDrive/nytval2.csv', index=False)
pd.DataFrame(nyt_test).to_csv('/content/drive/MyDrive/nyttest2.csv', index=False)

In [ ]:
#validation embeddings

val_emb = []

for i in range(len(val_answers)):
  print(i)
  val_emb.append(model.encode([val_clues[i],val_answers[i]]))

Streaming output truncated to the last 5000 lines.
23721
23722
23723
23724
23725
23726
23727
23728
23729
23730
23731
23732
23733
23734
23735
23736
23737
23738
23739
23740
23741
23742
23743
23744
23745
23746
23747
23748
23749
23750
23751
23752
23753
23754
23755
23756
23757
23758
23759
23760
23761
23762
23763
23764
23765
23766
23767
23768
23769
23770
23771
23772
23773
23774
23775
23776
23777
23778
23779
23780
23781
23782
23783
23784
23785
23786
23787
23788
23789
23790
23791
23792
23793
23794
23795
23796
23797
23798
23799
23800
23801
23802
23803
23804
23805
23806
23807
23808
23809
23810
23811
23812
23813
23814
23815
23816
23817
23818
23819
23820
23821
23822
23823
23824
23825
23826
23827
23828
23829
23830
23831
23832
23833
23834
23835
23836
23837
23838
23839
23840
23841
23842
23843
23844
23845
23846
23847
23848
23849
23850
23851
23852
23853
23854
23855
23856
23857
23858
23859
23860
23861
23862
23863
23864
23865
23866
23867
23868
23869
23870
23871
23872
23873
23874
23875
23876
23877
23878
2

In [ ]:
#separate into clue/answers

val_clue_embed = np.array([val_emb[j][0] for j in range(len(val_emb))])
val_answer_embed = np.array([val_emb[j][1] for j in range(len(val_emb))])

In [ ]:
#test answer/clues

test_answers = np.array([k for j in [list(eval(i)) for i in nyt_test['answer']] for k in j])
test_clues = np.array([k for j in [list(eval(i)) for i in nyt_test['clue']] for k in j])

In [ ]:
#compute test embeddings

test_emb = []

for i in range(len(test_answers)):
  print(i)
  test_emb.append(model.encode([test_clues[i],test_answers[i]]))

Streaming output truncated to the last 5000 lines.
19736
19737
19738
19739
19740
19741
19742
19743
19744
19745
19746
19747
19748
19749
19750
19751
19752
19753
19754
19755
19756
19757
19758
19759
19760
19761
19762
19763
19764
19765
19766
19767
19768
19769
19770
19771
19772
19773
19774
19775
19776
19777
19778
19779
19780
19781
19782
19783
19784
19785
19786
19787
19788
19789
19790
19791
19792
19793
19794
19795
19796
19797
19798
19799
19800
19801
19802
19803
19804
19805
19806
19807
19808
19809
19810
19811
19812
19813
19814
19815
19816
19817
19818
19819
19820
19821
19822
19823
19824
19825
19826
19827
19828
19829
19830
19831
19832
19833
19834
19835
19836
19837
19838
19839
19840
19841
19842
19843
19844
19845
19846
19847
19848
19849
19850
19851
19852
19853
19854
19855
19856
19857
19858
19859
19860
19861
19862
19863
19864
19865
19866
19867
19868
19869
19870
19871
19872
19873
19874
19875
19876
19877
19878
19879
19880
19881
19882
19883
19884
19885
19886
19887
19888
19889
19890
19891
19892
19893
1

In [ ]:
#split test embeddings

test_clue_embed = np.array([test_emb[j][0] for j in range(len(test_emb))])
test_answer_embed = np.array([test_emb[j][1] for j in range(len(test_emb))])

In [ ]:
#validation/test clue embeddings to csvs

pd.DataFrame(val_clue_embed).to_csv('/content/drive/MyDrive/valclueembed2.csv', index=False)
pd.DataFrame(val_answer_embed).to_csv('/content/drive/MyDrive/valanswerembed2.csv', index=False)
pd.DataFrame(test_clue_embed).to_csv('/content/drive/MyDrive/testclueembed2.csv', index=False)
pd.DataFrame(test_answer_embed).to_csv('/content/drive/MyDrive/testanswerembed2.csv', index=False)